In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
import pandas as pd
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Yield Prediction")\
        .config('spark.ui.port','4050')\
        .getOrCreate()
spark

In [3]:
!git clone https://github.com/ronanmccormack-ca/ProgrammingforBigData.git

fatal: destination path 'ProgrammingforBigData' already exists and is not an empty directory.


In [45]:
df = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/inventory_data.csv',header='true',inferSchema=True)

In [114]:
df.groupBy("batch_id","work_order_type_name").sum('measure_event_value')\
.where((col("work_order_type_name") == 'Drying')).show()

+--------+--------------------+------------------------+
|batch_id|work_order_type_name|sum(measure_event_value)|
+--------+--------------------+------------------------+
|   22162|              Drying|                  405262|
|   15528|              Drying|                    6494|
|   15531|              Drying|                    5317|
|   15534|              Drying|                   14453|
|    9697|              Drying|                  772962|
|   15533|              Drying|                   10459|
|   15529|              Drying|                    5086|
|   15527|              Drying|                  287328|
|    7246|              Drying|                  369300|
|   15530|              Drying|                   11160|
|   16037|              Drying|                  350202|
|   15532|              Drying|                    6563|
+--------+--------------------+------------------------+



In [100]:
df.filter(col('batch_id')==9697).show()

+------------------+----------------+-------------------------------+--------+---------------+-----------+-------------+-----------+-------------+-------------+--------------------+------------------+--------------------+---------------------+---------------------+-----------------+--------------------+-------------------+------------------+----+----+----+----+----+----+----+
|measure_event_type|measure_event_id|measure_event_performed_on_date|batch_id|     batch_name|cultivar_id|cultivar_name|location_id|location_name|work_order_id|     work_order_name|work_order_type_id|work_order_type_name|work_order_start_date|work_order_close_date|substance_type_id| substance_type_name|measure_event_value|measure_event_unit|_c19|_c20|_c21|_c22|_c23|_c24|_c25|
+------------------+----------------+-------------------------------+--------+---------------+-----------+-------------+-----------+-------------+-------------+--------------------+------------------+--------------------+---------------------

In [115]:
df.filter(col('cultivar_name')=='Blue Lime Pi').show()

+------------------+----------------+-------------------------------+--------+--------------------+-----------+-------------+-----------+--------------------+-------------+--------------------+------------------+--------------------+---------------------+---------------------+-----------------+--------------------+-------------------+------------------+----+----+----+----+----+----+----+
|measure_event_type|measure_event_id|measure_event_performed_on_date|batch_id|          batch_name|cultivar_id|cultivar_name|location_id|       location_name|work_order_id|     work_order_name|work_order_type_id|work_order_type_name|work_order_start_date|work_order_close_date|substance_type_id| substance_type_name|measure_event_value|measure_event_unit|_c19|_c20|_c21|_c22|_c23|_c24|_c25|
+------------------+----------------+-------------------------------+--------+--------------------+-----------+-------------+-----------+--------------------+-------------+--------------------+------------------+------

In [127]:
df.filter((col('cultivar_name')=='Blue Lime Pi')&\
          (col('work_order_type_name')=='Drying')&\
          (col('substance_type_name')=='Dried Cannabis')\
          &(col('measure_event_type')=='WO - OUTPUT')).show()

+------------------+----------------+-------------------------------+--------+--------------------+-----------+-------------+-----------+-------------+-------------+--------------------+------------------+--------------------+---------------------+---------------------+-----------------+-------------------+-------------------+------------------+----+----+----+----+----+----+----+
|measure_event_type|measure_event_id|measure_event_performed_on_date|batch_id|          batch_name|cultivar_id|cultivar_name|location_id|location_name|work_order_id|     work_order_name|work_order_type_id|work_order_type_name|work_order_start_date|work_order_close_date|substance_type_id|substance_type_name|measure_event_value|measure_event_unit|_c19|_c20|_c21|_c22|_c23|_c24|_c25|
+------------------+----------------+-------------------------------+--------+--------------------+-----------+-------------+-----------+-------------+-------------+--------------------+------------------+--------------------+--------

In [128]:
co2_sensor_data = spark.read.csv('/content/ProgrammingforBigData/Big Data Project/data/co2_sensor_1.csv',header='true',inferSchema=True)

In [138]:
co2_sensor_data.filter((col('Date')<='2021-07-15')&(col('Date')>='2021-04-15')).show()

+--------------------+--------+------------------+-------------------+-----+--------------------+-------+-------------+------------+---------+----------+---------------+-------+
|     DataMessageGUID|SensorID|       Sensor Name|               Date|Value|     Formatted Value|Battery|     Raw Data|Sensor State|GatewayID|Alert Sent|Signal Strength|Voltage|
+--------------------+--------+------------------+-------------------+-----+--------------------+-------+-------------+------------+---------+----------+---------------+-------+
|54aff7bb-d97c-4ec...|  595683|CO2 Meter - 595683|2021-04-29 13:26:00|  512|CO2 Instantaneous...|     73|512|541|0|359|           0|   942254|     false|             71|   2.64|
|e2c27bad-fc36-4ca...|  595683|CO2 Meter - 595683|2021-04-29 13:16:00|  522|CO2 Instantaneous...|     73|522|541|0|383|           0|   942254|     false|             67|   2.64|
|d186c2b5-0453-40e...|  595683|CO2 Meter - 595683|2021-04-29 13:06:00|  520|CO2 Instantaneous...|     73|520|5